In [1]:
from datasets import load_dataset,Dataset,concatenate_datasets,interleave_datasets

/home/infidea/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

mmmlu_origin = load_dataset('overfit-brothers/mmmlu_kor_revised_original')

# 선택지 순서 셔플

In [15]:
from datasets import DatasetDict, Dataset
import random
from copy import deepcopy

def augment_dataset(dataset_dict, num_augmentations):
    # 새로운 DatasetDict 생성
    augmented_dataset = DatasetDict()
    
    # 각 스플릿에 대해 증강 수행
    for split in dataset_dict.keys():
        original_data = dataset_dict[split]
        augmented_examples = []
        
        # 각 예제에 대해 증강 수행
        for example in original_data:
            
            # 추가 증강 데이터 생성
            for _ in range(num_augmentations):
                new_example = deepcopy(example)
                
                # 현재 정답 찾기
                original_question = example['question_original']
                original_answer = example['answer']
                choices = ['A', 'B', 'C', 'D']
                choice_contents = {
                    'A': example['A'],
                    'B': example['B'],
                    'C': example['C'],
                    'D': example['D']
                }
                
                # 선택지 순서 섞기 (정답 위치가 바뀔 때까지)
                while True:
                    new_choices = choices.copy()
                    random.shuffle(new_choices)
                    if new_choices.index(original_answer) != choices.index(original_answer):
                        break
                
                # 새로운 순서로 선택지 재배치
                for i, new_choice in enumerate(new_choices):
                    new_example[choices[i]] = choice_contents[new_choice]
                    if new_choice == original_answer:
                        new_example['answer'] = choices[i]
                a,b,c,d = new_example['A'], new_example['B'], new_example['C'], new_example['D']
                new_example['question'] =  f'{original_question} ### 선택지: A.{a} B.{b} C.{c} D.{d}'
                choices = []
                for choice in new_example.keys():
                    if choice not in ['question_original', 'answer', 'question']:
                        choices.append(new_example[choice])
                new_example['choices'] = choices
                del new_example['A']
                del new_example['B'] 
                del new_example['C']
                del new_example['D']                
                augmented_examples.append(new_example)
        
        # 증강된 데이터로 새로운 Dataset 생성
        augmented_dataset[split] = Dataset.from_list(augmented_examples)
    
    return augmented_dataset


In [ ]:

mmmlu_shuffle = augment_dataset(mmmlu_origin, 2)


In [20]:
mmmlu_shuffle['train'][0]

{'question': '다음 중 99% 신뢰구간을 가장 적절하게 정의한 것은 무엇입니까? ### 선택지: A.반복적인 표본에서 99%의 경우 그 구간에는 모수의 추정 값이 포함될 수 있습니다 B.반복적인 표본에서 99%의 경우 귀무 가설이 거짓일 때 해당 가설이 기각되지 않을 것입니다 C.반복적인 표본에서 99%의 경우 귀무 가설이 기각될 것입니다 D.반복적인 표본에서 99%의 경우 그 구간에는 모수의 실제 값이 포함될 수 있습니다',
 'answer': 'D',
 'question_original': '다음 중 99% 신뢰구간을 가장 적절하게 정의한 것은 무엇입니까?',
 'choices': ['반복적인 표본에서 99%의 경우 그 구간에는 모수의 추정 값이 포함될 수 있습니다',
  '반복적인 표본에서 99%의 경우 귀무 가설이 거짓일 때 해당 가설이 기각되지 않을 것입니다',
  '반복적인 표본에서 99%의 경우 귀무 가설이 기각될 것입니다',
  '반복적인 표본에서 99%의 경우 그 구간에는 모수의 실제 값이 포함될 수 있습니다']}

In [12]:
mmmlu_shuffle['train'][0]

{'question': '다음 중 99% 신뢰구간을 가장 적절하게 정의한 것은 무엇입니까? ### 선택지: A.반복적인 표본에서 99%의 경우 그 구간에는 모수의 추정 값이 포함될 수 있습니다 B.반복적인 표본에서 99%의 경우 귀무 가설이 거짓일 때 해당 가설이 기각되지 않을 것입니다 C.반복적인 표본에서 99%의 경우 그 구간에는 모수의 실제 값이 포함될 수 있습니다 D.반복적인 표본에서 99%의 경우 귀무 가설이 기각될 것입니다',
 'answer': 'C',
 'question_original': '다음 중 99% 신뢰구간을 가장 적절하게 정의한 것은 무엇입니까?',
 'A': '반복적인 표본에서 99%의 경우 그 구간에는 모수의 추정 값이 포함될 수 있습니다',
 'B': '반복적인 표본에서 99%의 경우 귀무 가설이 거짓일 때 해당 가설이 기각되지 않을 것입니다',
 'C': '반복적인 표본에서 99%의 경우 그 구간에는 모수의 실제 값이 포함될 수 있습니다',
 'D': '반복적인 표본에서 99%의 경우 귀무 가설이 기각될 것입니다'}

# 선택지 줄이기

In [23]:
from datasets import DatasetDict, Dataset
import random
from copy import deepcopy

def reduce_choices(dataset_dict):
    augmented_dataset = DatasetDict()
    
    for split in dataset_dict.keys():
        original_data = dataset_dict[split]
        reduced_examples = []
        
        for example in original_data:
            new_example = deepcopy(example)
            original_question = example['question'].split('###')[0].strip()
            
            # 현재 정답과 선택지들
            answer = example['answer']
            choices = ['A', 'B', 'C', 'D']
            choice_contents = {
                'A': example['A'],
                'B': example['B'],
                'C': example['C'],
                'D': example['D']
            }
            
            # 제거할 선택지 개수를 랜덤하게 결정 (1 또는 2)
            num_to_remove = random.randint(1, 2)
            
            # 정답을 제외한 선택지 중에서 랜덤하게 선택하여 제거
            removable_choices = [c for c in choices if c != answer]
            choices_to_remove = random.sample(removable_choices, num_to_remove)
            
            # 새로운 선택지 순서 생성
            remaining_choices = [c for c in choices if c not in choices_to_remove]
            new_choice_contents = {}
            new_answer = answer
            
            # 선택지 재배치
            new_choice_labels = ['A', 'B', 'C'][:len(remaining_choices)]
            for i, new_choice in enumerate(new_choice_labels):
                original_choice = remaining_choices[i]
                new_choice_contents[new_choice] = choice_contents[original_choice]
                if original_choice == answer:
                    new_answer = new_choice
            
            # 새로운 예제 생성
            new_example['answer'] = new_answer
            for i, label in enumerate(new_choice_labels):
                new_example[label] = new_choice_contents[label]
            
            # 제거된 선택지들 삭제
            for choice in choices[len(remaining_choices):]:
                del new_example[choice]
            
            # question 칼럼 업데이트
            choice_text = ' '.join([f"{label}.{new_choice_contents[label]}" for label in new_choice_labels])
            new_example['question'] = f"{original_question} ### 선택지: {choice_text}"
            
            # None 값을 가진 키 제거
            new_example = {k: v for k, v in new_example.items() if v is not None}
            choices = []
            for choice in new_example.keys():
                if choice not in ['question_original', 'answer', 'question']:
                    choices.append(new_example[choice])
            new_example['choices'] = choices
            
            for key in ['A', 'B', 'C', 'D']:
                if key in new_example:
                    del new_example[key]
            reduced_examples.append(new_example)
        
        augmented_dataset[split] = Dataset.from_list(reduced_examples)
    
    return augmented_dataset


In [ ]:

mmmlu_reduced = reduce_choices(mmmlu_origin)

# 선택지 늘리기

In [ ]:
GPT_API_KEY = "sk-XXX"
import openai
client = openai.OpenAI(api_key=GPT_API_KEY)

In [29]:
def make_prompt(question, choices, answer, num_augment):
    prompt = f"""
    문제와 선택지를 읽고 파악한 후에, 정답이 아닌 추가적인 선택지를 {num_augment}개 만들어 주세요. 
    
    - 요구사항: 
    1. 주어진 선택지들과 전부 다른 선택지를 만들어야 합니다.
    2. 문제와 관련이 있는 선택지를 만들어야 합니다.
    3. 정답과는 논리적으로 연관이 없는 선택지를 만들어야 합니다.
    4. 알파벳이나 숫자 번호 없이 선택지만 출력해야 합니다.
    
    문제: {question}
    선택지: {choices}
    정답: {answer}
    
    출력 형식(리스트):
    [
        "추가적인 선택지 1",
        "추가적인 선택지 2",
        "추가적인 선택지 3",
        ...
    ]
    
    추가 선택지 리스트:
    """
    return prompt

In [30]:
from pydantic import BaseModel
from typing import List
class AnswerModel(BaseModel):
    answer : List[str]

In [ ]:
kmmlu_origin['train'][0]

{'question': '(주)한공은 2013년 10월 1일 장기투자를 목적으로 (주)더존의 주식 100주를 주당 1,000원에 취득하였다. (주)더존의 주식은 한국거래소에 상장되어 있으며, 2013년 12월 31일 결산일의 공정가치는 주당 1,300원이다. 공정가치를 반영하기 위한 분개로 옳은 것은? ### 선택지: A.(차) 단기매매증권 : 30,000원, (대) 단기매매증권평가이익 : 30,000원 B.(차) 매도가능증권 : 30,000원, (대) 매도가능증권평가이익 : 30,000원 C.(차) 단기매매증권평가손실 : 30,000원, (대) 단기매매증권 : 30,000원 D.(차)  매도가능증권평가손실 : 30,000원, (대) 매도가능증권 : 30,000원',
 'answer': 'B',
 'question_original': '(주)한공은 2013년 10월 1일 장기투자를 목적으로 (주)더존의 주식 100주를 주당 1,000원에 취득하였다. (주)더존의 주식은 한국거래소에 상장되어 있으며, 2013년 12월 31일 결산일의 공정가치는 주당 1,300원이다. 공정가치를 반영하기 위한 분개로 옳은 것은?',
 'A': '(차) 단기매매증권 : 30,000원, (대) 단기매매증권평가이익 : 30,000원',
 'B': '(차) 매도가능증권 : 30,000원, (대) 매도가능증권평가이익 : 30,000원',
 'C': '(차) 단기매매증권평가손실 : 30,000원, (대) 단기매매증권 : 30,000원',
 'D': '(차)  매도가능증권평가손실 : 30,000원, (대) 매도가능증권 : 30,000원'}

In [37]:
from datasets import DatasetDict, Dataset
import random
from copy import deepcopy

def augment_choices(dataset_dict):
    augmented_dataset = DatasetDict()
    
    for split in dataset_dict.keys():
        original_data = dataset_dict[split]
        auemented_examples = []
        
        for example in original_data:
            new_example = deepcopy(example)
            original_question = example['question_original']
            
            # 현재 정답과 선택지들
            answer = example['answer']
            original_choices = ['A', 'B', 'C', 'D']
            choice_contents = {
                'A': example['A'],
                'B': example['B'],
                'C': example['C'],
                'D': example['D']
            }
            
            num_to_augment = random.randint(1, 3)
            prompt = make_prompt(original_question, str(choice_contents), answer, num_to_augment)
            response = client.beta.chat.completions.parse(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ],
                response_format=AnswerModel
            )
            answer_list = eval(response.choices[0].message.content)['answer']
            
            # Add new choices in alphabetical order
            maked_choices = ['E', 'F', 'G'][:num_to_augment]
            for maked_choice, new_content in zip(maked_choices, answer_list):
                new_example[maked_choice] = new_content            
            
            # Shuffle all choices
            all_choices = original_choices + maked_choices
            choice_mapping = {}
            shuffled_contents = []
            
            # Create list of all contents
            for choice in all_choices:
                shuffled_contents.append(new_example[choice])
                if choice == answer:
                    original_answer_content = new_example[choice]
            
            # Shuffle contents
            random.shuffle(shuffled_contents)
            
            # Reassign shuffled contents and track new answer position
            for i, choice in enumerate(all_choices):
                new_example[choice] = shuffled_contents[i]
                if shuffled_contents[i] == original_answer_content:
                    new_example['answer'] = choice
                    
            full_choices = original_choices+maked_choices
            # question 칼럼 업데이트
            choice_text = ' '.join([f"{label}.{new_example[label]}" for label in full_choices])
            new_example['question'] = f"{original_question} ### 선택지: {choice_text}"
            
            choices = []
            for choice in new_example.keys():
                if choice not in ['question_original', 'answer', 'question']:
                    choices.append(new_example[choice])
            new_example['choices'] = choices
            
            for key in ['A', 'B', 'C', 'D','E', 'F', 'G']:
                if key in new_example:
                    del new_example[key]
            auemented_examples.append(new_example)
        augmented_dataset[split] = Dataset.from_list(auemented_examples)
    return augmented_dataset

In [ ]:

mmmlu_augmented = augment_choices(mmmlu_origin)

In [40]:
mmmlu_augmented['train'][0]

{'question': '다음 중 99% 신뢰구간을 가장 적절하게 정의한 것은 무엇입니까? ### 선택지: A.반복적인 표본에서 99%의 경우 귀무 가설이 기각될 것입니다 B.반복적인 표본에서 99%의 경우 귀무 가설이 거짓일 때 해당 가설이 기각되지 않을 것입니다 C.반복적인 표본에서 99%의 경우 통계량이 항상 같은 값을 가질 것입니다 D.반복적인 표본에서 99%의 경우 그 구간에는 모수의 실제 값이 포함될 수 있습니다 E.반복적인 표본에서 99%의 경우 구간의 길이가 일정하게 유지될 것입니다 F.반복적인 표본에서 99%의 경우 그 구간에는 모수의 추정 값이 포함될 수 있습니다',
 'answer': 'D',
 'question_original': '다음 중 99% 신뢰구간을 가장 적절하게 정의한 것은 무엇입니까?',
 'choices': ['반복적인 표본에서 99%의 경우 귀무 가설이 기각될 것입니다',
  '반복적인 표본에서 99%의 경우 귀무 가설이 거짓일 때 해당 가설이 기각되지 않을 것입니다',
  '반복적인 표본에서 99%의 경우 통계량이 항상 같은 값을 가질 것입니다',
  '반복적인 표본에서 99%의 경우 그 구간에는 모수의 실제 값이 포함될 수 있습니다',
  '반복적인 표본에서 99%의 경우 구간의 길이가 일정하게 유지될 것입니다',
  '반복적인 표본에서 99%의 경우 그 구간에는 모수의 추정 값이 포함될 수 있습니다']}

In [43]:
def transform_data(example):
    choices = [
        example['A'],
        example['B'], 
        example['C'],
        example['D']
    ]
    
    # 기존 A,B,C,D 키 제거
    del example['A']
    del example['B']
    del example['C'] 
    del example['D']
    
    # choices 칼럼 추가
    example['choices'] = choices
    
    return example

In [44]:
kmmlu_origin['train'] = kmmlu_origin['train'].map(transform_data)

Map: 100%|██████████| 379/379 [00:00<00:00, 9930.48 examples/s]


In [ ]:
mmmlu_origin['train'] = mmmlu_origin['train'].map(transform_data)

In [ ]:

mmmlu_full = concatenate_datasets([mmmlu_origin['train'],mmmlu_shuffle['train'], mmmlu_reduced['train'],mmmlu_augmented['train']])

In [55]:
mmmlu_full[0]

{'question': '다음 중 99% 신뢰구간을 가장 적절하게 정의한 것은 무엇입니까? ### 선택지: A.반복적인 표본에서 99%의 경우 그 구간에는 모수의 실제 값이 포함될 수 있습니다 B.반복적인 표본에서 99%의 경우 그 구간에는 모수의 추정 값이 포함될 수 있습니다 C.반복적인 표본에서 99%의 경우 귀무 가설이 기각될 것입니다 D.반복적인 표본에서 99%의 경우 귀무 가설이 거짓일 때 해당 가설이 기각되지 않을 것입니다',
 'answer': 'A',
 'question_original': '다음 중 99% 신뢰구간을 가장 적절하게 정의한 것은 무엇입니까?',
 'choices': ['반복적인 표본에서 99%의 경우 그 구간에는 모수의 실제 값이 포함될 수 있습니다',
  '반복적인 표본에서 99%의 경우 그 구간에는 모수의 추정 값이 포함될 수 있습니다',
  '반복적인 표본에서 99%의 경우 귀무 가설이 기각될 것입니다',
  '반복적인 표본에서 99%의 경우 귀무 가설이 거짓일 때 해당 가설이 기각되지 않을 것입니다']}

In [98]:
mmmlu_augmented['train'][121]

{'question': 'GDP는  ### 선택지: A.국가 국경 내에서 생산되는 것이다. B.국가의 문화적 영향을 반영한 경제 지표이다. C.해외에서 수입되는 모든 상품의 가치이다. D.생산 요소로 벌어들인 소득에 감가상각비 및 간접 사업세를 더한 금액이다. E.한 국가의 국민이 어디에 있든 생산할 수 있는 것이다. F.(A) 및 (C)',
 'answer': 'F',
 'question_original': 'GDP는 ',
 'A': '국가 국경 내에서 생산되는 것이다.',
 'B': '국가의 문화적 영향을 반영한 경제 지표이다.',
 'C': '해외에서 수입되는 모든 상품의 가치이다.',
 'D': '생산 요소로 벌어들인 소득에 감가상각비 및 간접 사업세를 더한 금액이다.',
 'E': '한 국가의 국민이 어디에 있든 생산할 수 있는 것이다.'}

In [ ]:

mmmlu_full.push_to_hub('overfit-brothers/mmmlu_revised_augmented',private=True)

# 풀이 만들기

In [ ]:
GPT_API_KEY = "sk-XXX"
import openai
client = openai.OpenAI(api_key=GPT_API_KEY)

In [6]:
def make_cot_prompt(question):
    prompt = f"""
    문제를 읽고 파악한 후에, 풀이과정과 정답을 알려주세요. 
    
    - 요구사항: 
    1. 문제를 읽고 파악하고 문제에서 요구하는 목표를 간략하게 1문장으로 말하세요.
    2. 풀이과정은 논리적 사고 흐름에 따라 생각의 사슬 형식으로 설명하세요. 
    3. 도출된 답안을 제시하고 마지막에 다음과 같이 정답의 기호를 한번 더 언급해 주세요. ### 정답: [정답 기호]
    
    답변 형식:
    1) 문제 분석: [문제의 핵심 내용 정리]
    2) 풀이 과정: [단계별 설명,문장 형식]
    3) 최종 정답: ### 정답: [정답 기호]
    
    문제: 
    {question}
    """
    return prompt

In [ ]:
from datasets import load_dataset,Dataset

mmmlu = load_dataset("overfit-brothers/mmmlu_revised_augmented")

In [ ]:
import boto3
import json
import os
from tqdm import tqdm
os.environ['AWS_ACCESS_KEY_ID'] = 'XXX'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'XXX'
def make_cot(example):
    # bedrock_runtime = boto3.client(
    #     service_name='bedrock-runtime',
    #     region_name='us-east-1'
    # )
    res_list = []
    total_count = len(example)
    answer_count = 0
    input_tokens = 0
    output_tokens = 0
    for datum in tqdm(example):
        question = datum['question']
        answer = datum['answer']
        prompt = make_cot_prompt(question)
        # body = json.dumps({
        # "anthropic_version": "bedrock-2023-05-31",
        # "max_tokens": 1000,
        # "messages": [
        #     {
        #         "role": "user",
        #         "content": prompt
        #             }
        #         ]
        #     })
        # # Claude API 호출
        # response = bedrock_runtime.invoke_model(
        #     modelId='us.anthropic.claude-3-5-sonnet-20241022-v2:0',
        #     body=body
        # )
        
        # 응답 파싱
        # response_body = json.loads(response.get('body').read())
        # response_text = response_body['content'][0]['text']
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=2048
        )
        input_tokens += response.usage.prompt_tokens
        output_tokens += response.usage.completion_tokens
        response_text = response.choices[0].message.content
        response_answer = max(['A','B','C','D','E','F','G'], key=lambda x: response_text.count(x))
        if answer == response_answer:
            answer_count += 1
            datum['answer'] = response_text
            res_list.append(datum)
        else:
            continue
    res_dataset = Dataset.from_list(res_list)
    print(f'{answer_count}/{total_count} {answer_count/total_count*100:.2f}%')
    print(f'Input tokens: {input_tokens} (${input_tokens * 2.5 / 1000000:.4f})')
    print(f'Output tokens: {output_tokens} (${output_tokens * 10.0 / 1000000:.4f})')
    return res_dataset

In [ ]:
mmmlu_cot = make_cot(mmmlu['train'])
mmmlu_cot.push_to_hub('overfit-brothers/mmmlu_revised_augmented_cot',private=True)